In [1]:
!pip install SentencePiece 
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.9 MB/s eta 0:00:00


In [2]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import json
from datetime import datetime

In [3]:
print("start")
model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(torch_device)
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(
    torch_device)
print("model loaded")

def get_response(input_text,num_return_sequences,num_beams):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=num_beams, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

start
cuda


model loaded


In [4]:
def create_pegasus_data(path, out_path):
  with open(out_path, "w") as out_f:
    with open(path) as f:
      for id_, row in enumerate(f):
        data = json.loads(row)
        new_data = {"label": data["label"],
                    "premise": get_response(data["premise"], 1, 10)[0],
                    "hypothesis": get_response(data["hypothesis"], 1, 10)[0],
                    }
        if id_ % 100 == 0:
          print(id_, datetime.now())
        json.dump(new_data, out_f)
        out_f.write('\n')

In [5]:
create_pegasus_data("multinli_test_matched.jsonl", "multinli_test_matched_pegasus.jsonl")
create_pegasus_data("multinli_test_mismatched.jsonl", "multinli_test_mismatched_pegasus.jsonl")

0 2023-05-05 17:44:02.433120
100 2023-05-05 17:45:53.816036
200 2023-05-05 17:47:35.071152
300 2023-05-05 17:49:20.800098
400 2023-05-05 17:51:07.050783
500 2023-05-05 17:52:56.771216
600 2023-05-05 17:54:39.795477
700 2023-05-05 17:56:23.728826
800 2023-05-05 17:58:10.742855
900 2023-05-05 18:00:02.532823
1000 2023-05-05 18:01:49.935902
1100 2023-05-05 18:03:39.616671
1200 2023-05-05 18:05:21.770935
1300 2023-05-05 18:07:12.956705
1400 2023-05-05 18:09:01.259214
1500 2023-05-05 18:10:44.670561
1600 2023-05-05 18:12:25.037111
1700 2023-05-05 18:14:05.685295
1800 2023-05-05 18:15:44.951113
1900 2023-05-05 18:17:26.157949
2000 2023-05-05 18:19:14.940459
2100 2023-05-05 18:21:06.335621
2200 2023-05-05 18:22:58.718247
2300 2023-05-05 18:24:48.720886
2400 2023-05-05 18:26:31.322327
2500 2023-05-05 18:28:21.268481
2600 2023-05-05 18:30:01.676425
2700 2023-05-05 18:31:47.712564
2800 2023-05-05 18:33:38.670647
2900 2023-05-05 18:35:31.973682
3000 2023-05-05 18:37:19.461466
3100 2023-05-05 18:3